# SQLite Python excercise

In [1]:
import sqlite3

## Intro

Create DB

In [2]:
# database represented as a 'connection'
# new database is created if it does not exist on given name
con = sqlite3.connect("movie.db")

Database interface is called 'cursor'

In [3]:
cur = con.cursor()

The cursor allows us to interact with the database using SQL

In [4]:
cur.execute("CREATE TABLE movie (title, year, score)")


Add data:

In [5]:
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")
con.commit() # changes must be committed

Query data:

In [6]:
res = cur.execute("SELECT * FROM movie")
res.fetchall()

[('Monty Python and the Holy Grail', 1975, 8.2),
 ('And Now for Something Completely Different', 1971, 7.5)]

You can also add multiple rows using a one-liner:

In [7]:
data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
con.commit()

Notice that ? placeholders are used to bind data to the query. Always use placeholders instead of string formatting to bind Python values to SQL statements, to avoid SQL injection attacks.

What is an SQL injection attack? Consider the following:

In [8]:
# let's imagine we ask a user for their information. Instead
# they give us the following data:
row = "('dmbfkl', 1,1);DROP TABLE movie"

cur.executescript(f"INSERT INTO movie VALUES {row}")

The 'DROP TABLE' injection has destroyed the table

In [9]:
try:
    cur.execute("SELECT * FROM movie")
except Exception as e:
    print(e)


no such table: movie


## Excercise 1: load xlsx and store rows in a single sqlite table

Data: Hus ostolaskut 22Q1

In [10]:
import pandas as pd

In [11]:
# read only 1000 rows, because the file is huge!
pd.read_excel("https://www.hus.fi/sites/default/files/2022-04/husin-ostolaskutiedot-q1-2022.xlsx", nrows = 1000)

,Nimi,Y-tunnus,Tulosyksikkö,Tulosyksikön nimi,Vastuuyksikkö,Vastuuyksikön nimi,Palveluluokka,Laskun päiväys,Tosite numero,Ostotilaus numero,...,Tiliryhmän nimi,Tilin numero,Tilin nimi,Toimittajan nimi,Toimittajan maakoodi,Toimittajan y-tunnus,ALV-rekisterinumero,Netto summa,Alv summa,Kokonais summa
0,HUS-kuntayhtymä,1567535-0,700.0,TILAKESKUKSEN HALLINTO,7002018,"Vuokratilat, Helsinki",5501.0,24.03.2022,26462,NaN,...,"LÄMMITYS, SÄHKÖ, KAASU JA VESI",457020,Sähkö,KIINTEISTÖ OY PASILANRAITIO 5,FI,0550836-4,FI05508364,290.24,69.66,359.90
1,HUS-kuntayhtymä,1567535-0,710.0,YHTYMÄHALLINTO,7103002,Tasekirjaukset,9999.0,24.03.2022,26462,NaN,...,"MUUT SAAMISET, LYHYTAIKAISET",176024,"Alv-saamiset, palautus 24%",KIINTEISTÖ OY PASILANRAITIO 5,FI,0550836-4,FI05508364,NaN,69.66,69.66
2,HUS-kuntayhtymä,1567535-0,700.0,TILAKESKUKSEN HALLINTO,7002018,"Vuokratilat, Helsinki",5501.0,01.01.2022,23256,NaN,...,VUOKRAT,480020,Toimitilojen vuokrat,HENRY INVESTMENT OY,FI,2922998-4,FI29229984,14627.13,3510.51,18137.64
3,HUS-kuntayhtymä,1567535-0,710.0,YHTYMÄHALLINTO,7103002,Tasekirjaukset,9999.0,01.01.2022,23256,NaN,...,"MUUT SAAMISET, LYHYTAIKAISET",176024,"Alv-saamiset, palautus 24%",HENRY INVESTMENT OY,FI,2922998-4,FI29229984,NaN,3510.51,3510.51
4,HUS-kuntayhtymä,1567535-0,700.0,TILAKESKUKSEN HALLINTO,7002018,"Vuokratilat, Helsinki",5501.0,01.01.2022,23256,NaN,...,VUOKRAT,480020,Toimitilojen vuokrat,HENRY INVESTMENT OY,FI,2922998-4,FI29229984,7300.00,1752.00,9052.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,HUS-kuntayhtymä,1567535-0,760.0,HUS LOGISTIIKKA,7602014,"Varasto, logistiikkakeskus",5501.0,19.01.2022,24175,392217834.0,...,LYHYTAIKAISET SIIRTOVELAT,257070,Varaston siirtovelka,STERIPOLAR OY,FI,0603924-8,FI06039248,900.00,216.00,1116.00
996,HUS-kuntayhtymä,1567535-0,710.0,YHTYMÄHALLINTO,7103002,Tasekirjaukset,9999.0,19.01.2022,24175,NaN,...,"MUUT SAAMISET, LYHYTAIKAISET",176024,"Alv-saamiset, palautus 24%",STERIPOLAR OY,FI,0603924-8,FI06039248,NaN,216.00,216.00
997,HUS-kuntayhtymä,1567535-0,760.0,HUS LOGISTIIKKA,7602014,"Varasto, logistiikkakeskus",5501.0,19.01.2022,24049,392218055.0,...,LYHYTAIKAISET SIIRTOVELAT,257070,Varaston siirtovelka,STERIPOLAR OY,FI,0603924-8,FI06039248,19.50,4.68,24.18
998,HUS-kuntayhtymä,1567535-0,760.0,HUS LOGISTIIKKA,7602014,"Varasto, logistiikkakeskus",5501.0,19.01.2022,24049,392218055.0,...,LYHYTAIKAISET SIIRTOVELAT,257070,Varaston siirtovelka,STERIPOLAR OY,FI,0603924-8,FI06039248,650.00,156.00,806.00


In [12]:
#  your code here
df = pd.read_excel("https://www.hus.fi/sites/default/files/2022-04/husin-ostolaskutiedot-q1-2022.xlsx", nrows = 1000)
display(df.head(3))
display(df.columns)
print(f"Number of columns in data: {len(df.columns)}")

cur.execute("DROP TABLE IF EXISTS ostolasku")
cur.execute("CREATE TABLE ostolasku (Nimi, Y_tunnus, Tulosyksikko, Tulosyksikon_nimi, Vastuuyksikko, Vastuuyksikon_nimi,\
                                     Palveluluokka, Laskun_paivays, Tosite_numero, Ostotilaus numero, Sopimus_numero,\
                                     Tosite_paivamaara, Laskun_numero, Tiliryhma, Tiliryhman_nimi, Tilin_numero, Tilin_nimi,\
                                     Toimittajan_nimi, Toimittajan_maakoodi, Toimittajan_y_tunnus, ALV_rekisterinumero,\
                                     Netto_summa, Alv_summa, Kokonaissumma)")
cur.executemany("INSERT INTO ostolasku VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", df.values.tolist())
con.commit()

res = cur.execute("SELECT * FROM ostolasku")
res.fetchall()

,Nimi,Y-tunnus,Tulosyksikkö,Tulosyksikön nimi,Vastuuyksikkö,Vastuuyksikön nimi,Palveluluokka,Laskun päiväys,Tosite numero,Ostotilaus numero,...,Tiliryhmän nimi,Tilin numero,Tilin nimi,Toimittajan nimi,Toimittajan maakoodi,Toimittajan y-tunnus,ALV-rekisterinumero,Netto summa,Alv summa,Kokonais summa
0,HUS-kuntayhtymä,1567535-0,700.0,TILAKESKUKSEN HALLINTO,7002018,"Vuokratilat, Helsinki",5501.0,24.03.2022,26462,NaN,...,"LÄMMITYS, SÄHKÖ, KAASU JA VESI",457020,Sähkö,KIINTEISTÖ OY PASILANRAITIO 5,FI,0550836-4,FI05508364,290.24,69.66,359.90
1,HUS-kuntayhtymä,1567535-0,710.0,YHTYMÄHALLINTO,7103002,Tasekirjaukset,9999.0,24.03.2022,26462,NaN,...,"MUUT SAAMISET, LYHYTAIKAISET",176024,"Alv-saamiset, palautus 24%",KIINTEISTÖ OY PASILANRAITIO 5,FI,0550836-4,FI05508364,NaN,69.66,69.66
2,HUS-kuntayhtymä,1567535-0,700.0,TILAKESKUKSEN HALLINTO,7002018,"Vuokratilat, Helsinki",5501.0,01.01.2022,23256,NaN,...,VUOKRAT,480020,Toimitilojen vuokrat,HENRY INVESTMENT OY,FI,2922998-4,FI29229984,14627.13,3510.51,18137.64


Index(['Nimi ', 'Y-tunnus', 'Tulosyksikkö', 'Tulosyksikön nimi',
       'Vastuuyksikkö', 'Vastuuyksikön nimi', 'Palveluluokka',
       'Laskun päiväys', 'Tosite numero', 'Ostotilaus numero',
       'Sopimus numero', 'Tosite päivämäärä', 'Laskun numero', 'Tiliryhmä',
       'Tiliryhmän nimi', 'Tilin numero', 'Tilin nimi', 'Toimittajan nimi',
       'Toimittajan maakoodi', 'Toimittajan        y-tunnus',
       'ALV-rekisterinumero', 'Netto summa', 'Alv summa', 'Kokonais summa'],
      dtype='object')

Number of columns in data: 24


[('HUS-kuntayhtymä',
  '1567535-0',
  700.0,
  'TILAKESKUKSEN HALLINTO',
  7002018,
  'Vuokratilat, Helsinki',
  5501.0,
  '24.03.2022',
  26462,
  None,
  None,
  '24.03.2022',
  '000000001074',
  'T457',
  'LÄMMITYS, SÄHKÖ, KAASU JA VESI',
  457020,
  'Sähkö',
  'KIINTEISTÖ OY PASILANRAITIO 5',
  'FI',
  '0550836-4',
  'FI05508364',
  290.24,
  69.66,
  359.9),
 ('HUS-kuntayhtymä',
  '1567535-0',
  710.0,
  'YHTYMÄHALLINTO',
  7103002,
  'Tasekirjaukset',
  9999.0,
  '24.03.2022',
  26462,
  None,
  None,
  '24.03.2022',
  '000000001074',
  'A1760',
  'MUUT SAAMISET, LYHYTAIKAISET',
  176024,
  'Alv-saamiset, palautus 24%',
  'KIINTEISTÖ OY PASILANRAITIO 5',
  'FI',
  '0550836-4',
  'FI05508364',
  None,
  69.66,
  69.66),
 ('HUS-kuntayhtymä',
  '1567535-0',
  700.0,
  'TILAKESKUKSEN HALLINTO',
  7002018,
  'Vuokratilat, Helsinki',
  5501.0,
  '01.01.2022',
  23256,
  None,
  None,
  '01.01.2022',
  '000000008927',
  'T480',
  'VUOKRAT',
  480020,
  'Toimitilojen vuokrat',
  'HENRY I

## Exercise 2: There is a change! 

A provider called 'HENRY INVESTMENT OY' changes their company name to 'HENRY CAPITAL OY'. Perform the change on the records in the database. 

In [13]:
# your code here

old_name = "HENRY INVESTMENT OY"
new_name = "HENRY CAPITAL OY"

print(f"*****  NUMBER OF ROWS WITH NAME '{new_name}' BEFORE UPDATE *****")
res = cur.execute("SELECT COUNT(*) FROM ostolasku WHERE Toimittajan_nimi = ?", (new_name,))
print(res.fetchall())

update_sql = "UPDATE ostolasku SET Toimittajan_nimi = ? WHERE Toimittajan_nimi = ?"
cur.execute(update_sql, (new_name, old_name))
con.commit()

res = cur.execute("select changes(),total_changes()");
print(f"\n\n... {res.fetchall()[0][0]} rows updated!\n\n")

print(f"*****  NUMBER OF ROWS WITH NAME '{new_name}' AFTER UPDATE *****")
res = cur.execute("SELECT COUNT(*) FROM ostolasku WHERE Toimittajan_nimi = ?", (new_name,))
display(res.fetchall())


*****  NUMBER OF ROWS WITH NAME 'HENRY CAPITAL OY' BEFORE UPDATE *****
[(0,)]


... 12 rows updated!


*****  NUMBER OF ROWS WITH NAME 'HENRY CAPITAL OY' AFTER UPDATE *****


[(12,)]

## Exercise 3: Time for normalization!

From the previous example, you may have noticed how impractical it is to have all data in single master table. There is a lot of duplicate information risking breaking the data in face of change, and all operations need to handle the whole table and are thus very expensive. This is why normalization is needed. 

Normalization means breaking the table into entities and relations. The fundamental principle is to write each unit of information only once in the database, except for keys (IDs). There are exceptions, but this is what the aim is, usually. 

First, perform entity relationship modelling of the table (pen-and-paper or for example using https://erdplus.com). What are primary and foreign keys, attributes and constraints for the tables? Are the relations one-to-one or one-to-many?

Then, create and fill the tables according the ER model.

In [14]:
# your code here

display(df.iloc[:5, 6:].head(3))
#display(df['Tulosyksikkö'].isna().sum())
#display(df['Tulosyksikkö'].drop_duplicates())

# CREATE AND FILL TABLE organisaatio
cur.execute("DROP TABLE IF EXISTS organisaatio")
cur.execute("CREATE TABLE organisaatio (organisaatio_y_tunnus VARCHAR (9) PRIMARY KEY, organisaatio_nimi VARCHAR)")
con.commit()
cur.executemany("INSERT INTO organisaatio VALUES(?, ?)", df.loc[:, ["Y-tunnus","Nimi "]].drop_duplicates().values.tolist())
con.commit()
res = cur.execute("SELECT * FROM organisaatio")
display(res.fetchall())
res = cur.execute("SELECT count(*) FROM organisaatio")
print (f"*****  ORGANISAATIO: {res.fetchall()[0][0]} rows inserted  *****\n\n")

# CREATE AND FILL TABLE tulosyksikko
cur.execute("DROP TABLE IF EXISTS tulosyksikko")
cur.execute("CREATE TABLE tulosyksikko (tulosyksikko_id REAL PRIMARY KEY, organisaatio VARCHAR (9) REFERENCES organisaatio_y_tunnus, tulosyksikko_nimi VARCHAR)")
con.commit()
cur.executemany("INSERT INTO tulosyksikko VALUES(?, ?, ?)", df.loc[:, ["Tulosyksikkö", 'Y-tunnus', "Tulosyksikön nimi"]].drop_duplicates('Tulosyksikkö').values.tolist())
con.commit()
res = cur.execute("SELECT * FROM tulosyksikko LIMIT 3")
display(res.fetchall())
res = cur.execute("SELECT count(*) FROM tulosyksikko")
print (f"*****  TULOSYKSIKKÖ: {res.fetchall()[0][0]} rows inserted  *****\n\n")

# CREATE AND FILL TABLE vastuuyksikko
cur.execute("DROP TABLE IF EXISTS vastuuyksikko")
cur.execute("CREATE TABLE vastuuyksikko (vastuuyksikko_id INT PRIMARY KEY, organisaatio REFERENCES organisaatio_y_tunnus, vastuuyksikko_nimi VARCHAR)")
con.commit()
cur.executemany("INSERT INTO vastuuyksikko VALUES(?, ?, ?)", df.loc[:, ["Vastuuyksikkö", 'Y-tunnus', "Vastuuyksikön nimi"]].drop_duplicates('Vastuuyksikkö').values.tolist())
con.commit()
res = cur.execute("SELECT * FROM vastuuyksikko LIMIT 3")
display(res.fetchall())
res = cur.execute("SELECT count(*) FROM vastuuyksikko")
print (f"*****  VASTUUYKSIKKÖ: {res.fetchall()[0][0]} rows inserted  *****\n\n")

# CREATE AND FILL TABLE tiliryhma
cur.execute("DROP TABLE IF EXISTS tiliryhma")
cur.execute("CREATE TABLE tiliryhma (tiliryhma_id VARCHAR PRIMARY KEY, tiliryhma_nimi VARCHAR)")
con.commit()
cur.executemany("INSERT INTO tiliryhma VALUES(?, ?)", df.loc[:, ["Tiliryhmä", "Tiliryhmän nimi"]].drop_duplicates().values.tolist())
con.commit()
res = cur.execute("SELECT * FROM tiliryhma LIMIT 3")
display(res.fetchall())
res = cur.execute("SELECT count(*) FROM tiliryhma")
print (f"*****  TILIRYHMÄ: {res.fetchall()[0][0]} rows inserted  *****\n\n")

# CREATE AND FILL TABLE tili
cur.execute("DROP TABLE IF EXISTS tili")
cur.execute("CREATE TABLE tili (tilinumero INT PRIMARY KEY, tiliryhma REFERENCES tiliryhma_id, tili_nimi VARCHAR)")
con.commit()
cur.executemany("INSERT INTO tili VALUES(?, ?, ?)", df.loc[:, ["Tilin numero", 'Tiliryhmä', "Tilin nimi"]].drop_duplicates('Tilin numero').values.tolist())
con.commit()
res = cur.execute("SELECT * FROM tili LIMIT 3")
display(res.fetchall())
res = cur.execute("SELECT count(*) FROM tili")
print (f"*****  TILI: {res.fetchall()[0][0]} rows inserted  *****\n\n")

# CREATE AND FILL TABLE toimittaja
cur.execute("DROP TABLE IF EXISTS toimittaja")
cur.execute("CREATE TABLE toimittaja (toimittaja_y_tunnus VARCHAR(9) PRIMARY KEY, toimittaja_nimi VARCHAR, toimittaja_maakoodi VARCHAR(3))")
con.commit()
cur.executemany("INSERT INTO toimittaja VALUES(?, ?, ?)", df.loc[:, ["Toimittajan        y-tunnus", 'Toimittajan nimi', "Toimittajan maakoodi"]].drop_duplicates('Toimittajan        y-tunnus').values.tolist())
con.commit()
res = cur.execute("SELECT * FROM toimittaja LIMIT 3")
display(res.fetchall())
res = cur.execute("SELECT count(*) FROM toimittaja")
print (f"*****  TOIMITTAJA: {res.fetchall()[0][0]} rows inserted  *****\n\n")

# CREATE AND FILL TABLE tilaus
cur.execute("DROP TABLE IF EXISTS tilaus")
cur.execute("CREATE TABLE tilaus (tili REFERENCES tili_id, toimittaja REFERENCES toimittaja_y_tunnus, tulosyksikko REFERENCES tulosyksikko_id,\
             vastuuyksikko REFERENCES vastuuyksikko_id, tilaus_palveluluokka REAL, tilaus_laskunpaivays TEXT, tilaus_tositenumero INT,\
             tilaus_ostotilausnumero INT, tilaus_sopimusnumero INT, tilaus_tositepaivays TEXT, tilaus_laskunnumero INT, tilaus_alvnumero VARCHAR,\
             tilaus_netto_summa REAL, tilaus_alv_summa REAL, tilaus_kokonaissumma REAL)")
con.commit()
cur.executemany("INSERT INTO tilaus VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                df.loc[:, ["Tilin numero", 'Toimittajan        y-tunnus', "Tulosyksikkö", "Vastuuyksikkö", "Palveluluokka", "Laskun päiväys",
                "Tosite numero", "Ostotilaus numero", "Sopimus numero", "Tosite päivämäärä", "Laskun numero", "ALV-rekisterinumero",
                "Netto summa", "Alv summa", "Kokonais summa"]].values.tolist())
con.commit()
res = cur.execute("SELECT * FROM tilaus LIMIT 1")
display(res.fetchall())
res = cur.execute("SELECT count(*) FROM tilaus")
print (f"*****  TILAUS: {res.fetchall()[0][0]} rows inserted  *****\n\n")

,Palveluluokka,Laskun päiväys,Tosite numero,Ostotilaus numero,Sopimus numero,Tosite päivämäärä,Laskun numero,Tiliryhmä,Tiliryhmän nimi,Tilin numero,Tilin nimi,Toimittajan nimi,Toimittajan maakoodi,Toimittajan y-tunnus,ALV-rekisterinumero,Netto summa,Alv summa,Kokonais summa
0,5501.0,24.03.2022,26462,NaN,NaN,24.03.2022,000000001074,T457,"LÄMMITYS, SÄHKÖ, KAASU JA VESI",457020,Sähkö,KIINTEISTÖ OY PASILANRAITIO 5,FI,0550836-4,FI05508364,290.24,69.66,359.90
1,9999.0,24.03.2022,26462,NaN,NaN,24.03.2022,000000001074,A1760,"MUUT SAAMISET, LYHYTAIKAISET",176024,"Alv-saamiset, palautus 24%",KIINTEISTÖ OY PASILANRAITIO 5,FI,0550836-4,FI05508364,NaN,69.66,69.66
2,5501.0,01.01.2022,23256,NaN,NaN,01.01.2022,000000008927,T480,VUOKRAT,480020,Toimitilojen vuokrat,HENRY INVESTMENT OY,FI,2922998-4,FI29229984,14627.13,3510.51,18137.64


[('1567535-0', 'HUS-kuntayhtymä')]

*****  ORGANISAATIO: 1 rows inserted  *****




[(700.0, '1567535-0', 'TILAKESKUKSEN HALLINTO'),
 (710.0, '1567535-0', 'YHTYMÄHALLINTO'),
 (777.0, '1567535-0', 'MONIPALVELUALUE 5')]

*****  TULOSYKSIKKÖ: 28 rows inserted  *****




...

*****  TILI: 44 rows inserted  *****




[('0550836-4', 'KIINTEISTÖ OY PASILANRAITIO 5', 'FI'),
 ('2922998-4', 'HENRY INVESTMENT OY', 'FI'),
 ('2768040-5', 'TIKKURILA-GRILLI OY', 'FI')]

*****  TOIMITTAJA: 34 rows inserted  *****




[(457020,
  '0550836-4',
  700.0,
  7002018,
  5501.0,
  '24.03.2022',
  26462,
  None,
  None,
  '24.03.2022',
  1074,
  'FI05508364',
  290.24,
  69.66,
  359.9)]

*****  TILAUS: 1000 rows inserted  *****




In [18]:
#
# some validation for data tables and their integrity
#

# puuttuvia toimittaja tai tulosyksikkotietoja
print(f"{df['Toimittajan        y-tunnus'].isna().sum()} rivistä puuttuu toimittaja")
print(f"{df['Tulosyksikkö'].isna().sum()} rivistä puuttuu tulosyksikkö")

# Check that joining tables return correct number of rows
sql = "SELECT count (tilaus_tositenumero)\
       FROM tilaus, tili, tiliryhma, toimittaja, vastuuyksikko, tulosyksikko, organisaatio\
       WHERE tilaus.tili = tilinumero AND tili.tiliryhma = tiliryhma_id AND tilaus.toimittaja = toimittaja_y_tunnus\
       AND vastuuyksikko = vastuuyksikko_id AND tulosyksikko = tulosyksikko_id\
       AND tulosyksikko.organisaatio = organisaatio_y_tunnus AND vastuuyksikko.organisaatio = organisaatio_y_tunnus"
res = cur.execute(sql)
print(f"{res.fetchall()[0][0]} tietokannan muodostamassa liitoksessa täydelliset tiedot")

# test join query
sql = "SELECT tili, tili_nimi, tiliryhma_nimi, toimittaja_nimi, vastuuyksikko_nimi, tulosyksikko_nimi, organisaatio_nimi\
       FROM tilaus, tili, tiliryhma, toimittaja, vastuuyksikko, tulosyksikko, organisaatio\
       WHERE tili = tilinumero AND tiliryhma = tiliryhma_id AND toimittaja = toimittaja_y_tunnus\
       AND vastuuyksikko = vastuuyksikko_id AND tulosyksikko = tulosyksikko_id\
       AND tulosyksikko.organisaatio = organisaatio_y_tunnus AND vastuuyksikko.organisaatio = organisaatio_y_tunnus LIMIT 3"
res = cur.execute(sql)
display(res.fetchall())

11 rivistä puuttuu toimittaja
10 rivistä puuttuu tulosyksikkö
979 tietokannan muodostamassa liitoksessa täydelliset tiedot


[(457020,
  'Sähkö',
  'LÄMMITYS, SÄHKÖ, KAASU JA VESI',
  'KIINTEISTÖ OY PASILANRAITIO 5',
  'Vuokratilat, Helsinki',
  'TILAKESKUKSEN HALLINTO',
  'HUS-kuntayhtymä'),
 (176024,
  'Alv-saamiset, palautus 24%',
  'MUUT SAAMISET, LYHYTAIKAISET',
  'KIINTEISTÖ OY PASILANRAITIO 5',
  'Tasekirjaukset',
  'YHTYMÄHALLINTO',
  'HUS-kuntayhtymä'),
 (480020,
  'Toimitilojen vuokrat',
  'VUOKRAT',
  'HENRY CAPITAL OY',
  'Vuokratilat, Helsinki',
  'TILAKESKUKSEN HALLINTO',
  'HUS-kuntayhtymä')]

Finally, try to perform the change of name from 'HENRY INVESTMENT OY' to HENRY CAPITAL OY'. How many rows do you need to change?

In [16]:
# your code here

# change toimittajan nimi
update_sql = "UPDATE toimittaja SET toimittaja_nimi = ? WHERE toimittaja_nimi = ?"
cur.execute(update_sql, (new_name, old_name))
con.commit()

res = cur.execute("select changes(),total_changes()");
print(f"\n\n... {res.fetchall()[0][0]} rows updated!\n\n")





... 1 rows updated!




BONUS: using your normalized tables, for each unit ('tulosyksikkö') figure out which provider ('toimittaja') has billed highest cost during the month of January in 2022, and how much the total bill sum is.

# your code here

In [17]:
# query 5 biggest billers
sql = "SELECT toimittaja_nimi, CAST(SUM(tilaus_kokonaissumma) AS INT) AS maksettu\
       FROM tilaus, toimittaja\
       WHERE toimittaja = toimittaja_y_tunnus\
       GROUP BY toimittaja_nimi ORDER BY maksettu DESC LIMIT 5"
res = cur.execute(sql)
print ("*****  5 HIGHEST BILLERS IN ALL DATA  *****")
display(res.fetchall())

# query 5 biggest billers in january 2022
sql = "SELECT toimittaja_nimi, CAST(SUM(tilaus_kokonaissumma) AS INT) AS maksettu\
       FROM tilaus, toimittaja\
       WHERE toimittaja = toimittaja_y_tunnus AND tilaus_laskunpaivays LIKE '%01.2022'\
       GROUP BY toimittaja_nimi ORDER BY maksettu DESC LIMIT 5"
res = cur.execute(sql)
print ("\n\n*****  5 HIGHEST BILLERS IN JANUARY 2022  *****")
display(res.fetchall())

*****  5 HIGHEST BILLERS IN ALL DATA  *****


[('STERIPOLAR OY', 328028),
 ('BISNODE MARKETING OY', 272607),
 ('HENRY CAPITAL OY', 98865),
 ('VAASAN SAIRAANHOITOPIIRIN KUNTAYHTYMÄ', 40831),
 ('ELISA VIDERA OY', 8151)]



*****  5 HIGHEST BILLERS IN JANUARY 2022  *****


[('STERIPOLAR OY', 200016),
 ('BISNODE MARKETING OY', 77162),
 ('VAASAN SAIRAANHOITOPIIRIN KUNTAYHTYMÄ', 40732),
 ('HENRY CAPITAL OY', 32452),
 ('LEINONEN ARI', 3381)]